In [1]:
%env WANDB_API_KEY=32778d51a081d9a02605fc5461d7c8f19a4155f4
import wandb
wandb.login() 

env: WANDB_API_KEY=32778d51a081d9a02605fc5461d7c8f19a4155f4


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: trentini. Use `wandb login --relogin` to force relogin


True

In [4]:
import deepchem as dc

tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
[11:58:19] WARNING: not removing hydrogen atom without neighbors
[11:58:36] WARNING: not removing hydrogen atom without neighbors


In [5]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)

In [6]:
from deepchem.models.wandblogger import WandbLogger
from deepchem.models.callbacks import ValidationCallback

wandblogger = WandbLogger(project='deepchem_graphconv', name='basic')
vc_valid = ValidationCallback(valid_dataset, interval=100, metrics=[metric], transformers=transformers)

In [7]:
n_tasks = len(tasks)
model = dc.models.GraphConvModel(n_tasks, mode='classification', wandb_logger=wandblogger)
model.fit(train_dataset, nb_epoch=50, callbacks=[vc_valid])
wandblogger.finish()

AttributeError: module 'deepchem.models' has no attribute 'GraphConvModel'

In [8]:
train_score = model.evaluate(train_dataset, [metric], transformers)
valid_score = model.evaluate(valid_dataset, [metric], transformers)
test_score = model.evaluate(test_dataset, [metric], transformers)

print('Training set score:', train_score)
print('Validation set score:', valid_score)
print('Test set score:', test_score)

NameError: name 'model' is not defined

In [9]:
columns = ["run_name", "train", "val", "test"]
metrics_table = wandb.Table(columns=columns)

# Add a row for the Basic GCN
metrics_table.add_data("Basic GCN", train_score, valid_score, test_score)

NameError: name 'train_score' is not defined

In [10]:
from deepchem.models.layers import GraphConv, GraphPool, GraphGather
import tensorflow as tf
import tensorflow.keras.layers as layers

batch_size = 100

class MyGraphConvModel(tf.keras.Model):

  def __init__(self):
    super(MyGraphConvModel, self).__init__()
    self.gc1 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm1 = layers.BatchNormalization()
    self.gp1 = GraphPool()

    self.gc2 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm2 = layers.BatchNormalization()
    self.gp2 = GraphPool()

    self.dense1 = layers.Dense(256, activation=tf.nn.tanh)
    self.batch_norm3 = layers.BatchNormalization()
    self.readout = GraphGather(batch_size=batch_size, activation_fn=tf.nn.tanh)

    self.dense2 = layers.Dense(n_tasks*2)
    self.logits = layers.Reshape((n_tasks, 2))
    self.softmax = layers.Softmax()

  def call(self, inputs):
    gc1_output = self.gc1(inputs)
    batch_norm1_output = self.batch_norm1(gc1_output)
    gp1_output = self.gp1([batch_norm1_output] + inputs[1:])

    gc2_output = self.gc2([gp1_output] + inputs[1:])
    batch_norm2_output = self.batch_norm1(gc2_output)
    gp2_output = self.gp2([batch_norm2_output] + inputs[1:])

    dense1_output = self.dense1(gp2_output)
    batch_norm3_output = self.batch_norm3(dense1_output)
    readout_output = self.readout([batch_norm3_output] + inputs[1:])

    logits_output = self.logits(self.dense2(readout_output))
    return self.softmax(logits_output)

ModuleNotFoundError: No module named 'tensorflow'